# Current vs Density 

In [1]:
import sys
import os
sys.path.insert(0, '../magcolloids')

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from IPython.display import HTML, clear_output
import jsonpickle
import scipy.optimize as spo
import scipy.spatial as spa 
import magcolloids as mgc
import matplotlib as mpl
import string as st

from tqdm import tqdm_notebook

ureg = mgc.ureg

idx = pd.IndexSlice

%reload_ext autoreload
%autoreload 2

In [2]:
mpl.rc('text', usetex=True)
mpl.rcParams['figure.dpi'] = 150

In [3]:
directory = "Z:/Dimers/Simulations/DensityDependence3/"

In [4]:
h = 3.9 * ureg.um
tilt_angle = 7*ureg.deg
frequency = 3*ureg.Hz

In [5]:
packing = np.linspace(0.01, 0.9, 80)
packing

array([0.01      , 0.02126582, 0.03253165, 0.04379747, 0.05506329,
       0.06632911, 0.07759494, 0.08886076, 0.10012658, 0.11139241,
       0.12265823, 0.13392405, 0.14518987, 0.1564557 , 0.16772152,
       0.17898734, 0.19025316, 0.20151899, 0.21278481, 0.22405063,
       0.23531646, 0.24658228, 0.2578481 , 0.26911392, 0.28037975,
       0.29164557, 0.30291139, 0.31417722, 0.32544304, 0.33670886,
       0.34797468, 0.35924051, 0.37050633, 0.38177215, 0.39303797,
       0.4043038 , 0.41556962, 0.42683544, 0.43810127, 0.44936709,
       0.46063291, 0.47189873, 0.48316456, 0.49443038, 0.5056962 ,
       0.51696203, 0.52822785, 0.53949367, 0.55075949, 0.56202532,
       0.57329114, 0.58455696, 0.59582278, 0.60708861, 0.61835443,
       0.62962025, 0.64088608, 0.6521519 , 0.66341772, 0.67468354,
       0.68594937, 0.69721519, 0.70848101, 0.71974684, 0.73101266,
       0.74227848, 0.7535443 , 0.76481013, 0.77607595, 0.78734177,
       0.79860759, 0.80987342, 0.82113924, 0.83240506, 0.84367

In [6]:
def redefine_field(field,tilt_angle):
    field.fieldx = 'v_Bmag*(sin(v_freq*time*2*PI)*sin(v_theta)+cos(v_theta)*%s)'%np.sin(tilt_angle).magnitude
    field.fieldy = 'v_Bmag*(cos(v_freq*time*2*PI)*sin(v_theta))'%np.sin(tilt_angle).magnitude
    field.fieldz = 'v_Bmag*(cos(v_theta)*%s)'%np.cos(tilt_angle).magnitude
    return field

In [7]:
index_name = "index.dat"

import csv

def prepare_simulation(packing):
    
    region, positions = mgc.hexagonal_order(
        n_of_particles = 250,
        packing=packing, height = h.magnitude,
        r = 1.4, theta = np.pi/2)

    particles = mgc.particles(
        positions*ureg.um,
        radius = 1.4*ureg.um,
        susceptibility = 0.4,
        diffusion=0.073*ureg.um**2/ureg.s,
        density = 1000*ureg.kg/ureg.m**3,
        temperature=300*ureg.K)

    world = mgc.world(particles, temperature = 300*ureg.K,
                      lj_parameters = [1e-2*ureg.pg*ureg.um**2/ureg.us**2, 2**(-1/6)],
                      region=region*ureg.um, boundaries = ['p','p','f'], walls = [False,False,True],
                      dipole_cutoff = 30*ureg.um)

    field = mgc.field(magnitude = 7.28*ureg.mT, frequency = frequency,
                  angle = 27*ureg.degrees, multibody_iter=2)
    
    field = redefine_field(field,tilt_angle)

    sim = mgc.sim(dir_name = directory,
                  file_name="Phi_%s"%(packing),stamp_time=False,
                  timestep = 1e-4*ureg.s, framerate = 75*ureg.Hz, total_time = 60*ureg.s,
                  particles = particles, world = world, field = field, output=["x","y","z","mux","muy","muz","fx","fy"])
    
    sim.packing = packing
    
    return sim

In [8]:
simulations = [prepare_simulation(phi) for phi in packing]

In [9]:
sim = simulations[0]

In [10]:
def run_simulation(sim):
    
    import csv
    import os
    index_name = "index.dat"
    directory = sim.dir_name
    
    sim.generate_scripts()
    sim.run()
    
    packing = sim.packing
    
    with open(os.path.join(directory,index_name),'a',newline='') as file:
        writer = csv.writer(file,delimiter=' ')
        writer.writerow([os.path.split(sim.base_name)[-1], packing])

## Run asynchronously in several cores at once.

https://gist.github.com/aortiza/dd1eb47fa675c147bd44df3a98bc7a4e

In [11]:
from multiprocessing import Pool
import inspect

In [12]:
with open("dummy.py",'w') as file:
    file.write(inspect.getsource(run_simulation))

In [13]:
%%time
import dummy
if __name__ ==  '__main__': 
    num_processors = 8
    p=Pool(processes = num_processors)
    p.map(dummy.run_simulation,simulations)

Wall time: 9h 10min 33s
